# 10 - Gameplay and Decision Making

In [1]:
from constants import *
import numpy as np

In [2]:
# Both center of mass calculations only need to consider every other layer (the axis aligned ones)
# because the others are balanced regardless of which blocks are left
# In any axis aligned layer, blocks 1, 2, and 3 have centers at -1, 0, and 1 respectively if we
# place the origin and the z-axis at the center of the stack and use one block width as our unit distance
def compute_cm(stack, compute_x, bottom_layer = 0):
    assert bottom_layer < stack.shape[0], \
        f"The stack is too short ({stack.shape[0]}) for a bottom layer at {bottom_layer}"
    num_blocks = np.sum(stack[bottom_layer:])

    b = bottom_layer
    even, odd = (b + 1, b) if (b % 2) else (b, b + 1)
    # start on an even layer that goes in the x direction or an odd layer that goes in the y direction
    start_layer = even if compute_x else odd
    # skip non-aligned layers
    return np.sum(stack[start_layer::2] * BLOCK_POSITIONS) / num_blocks

def compute_x_cm(stack, bottom_layer = 0):
    return compute_cm(stack, compute_x = True, bottom_layer = bottom_layer)

def compute_y_cm(stack, bottom_layer = 0):
    return compute_cm(stack, compute_x = False, bottom_layer = bottom_layer)

In [7]:
def within_support_polygon(cm, layer):
    if np.all(layer == 0):
        return False
    idx = np.where(layer)[0]
    return LEFT_POSITIONS[idx[0]] <= cm <= RIGHT_POSITIONS[idx[-1]]

def validate_removals(stack, moves):
    x_cms = [compute_x_cm(stack, bottom_layer = b) for b in range(stack.shape[0])]
    y_cms = [compute_y_cm(stack, bottom_layer = b) for b in range(stack.shape[0])]
    blocks_above_layer = np.append(np.cumsum(np.sum(stack, axis = 1))[::-1], 0)[1:]

    output = []
    # ensure that the tower won't topple if the block is removed
    for move in possible_moves:
        block_layer, block_index = move
        valid_for_layers_below = True
        # check all layers below removed block to ensure the center of mass won't shift too much
        for layer_below in range(block_layer):
            # check x_cm if layer below is even and y_cm if layer below is odd
            old_cm = x_cms[layer_below + 1] if (layer_below % 2) else y_cms[layer_below + 1]
            n = blocks_above_layer[layer_below]
            new_cm = old_cm * (n / (n - 1))
            # if the axes are not aligned, then the contribution of the block to
            # CM is 0, but we will need to adjust if the axes are aligned
            if (block_layer % 2) == (layer_below % 2):
                new_cm -= (block_index - 1) / (n - 1)
            # the center of mass will fall outside the support polygon, so this move is invalid
            if not within_support_polygon(new_cm, stack[layer_below]):
                valid_for_layers_below = False
                break
        # skip over invalid move
        if not valid_for_layers_below:
            continue

        # finally ensure that there's no rug pull
        above_cm = (x_cms if (block_layer % 2) else y_cms)[block_layer + 1]
        # create layer without the candidate block
        layer = stack[block_layer].copy()
        layer[block_index] = 0
        # move has passed all tests
        if within_support_polygon(above_cm, layer):
            output.append(move)
    return output

def find_valid_moves(stack):
    height = stack.shape[0]
    # can't move blocks from top layer
    possible_moves = [(i, j) for (i, j) in np.ndindex(*stack.shape) if (stack[i, j] and i != height - 1)]

    # ensure that simply removing the block won't topple the stack
    possible_moves = validate_removals(stack, possible_moves)

    # add block to top of tower
    num_blocks_top_layer = np.sum(stack[-1])
    # add new layer
    if num_blocks_top_layer == 3:
        stack = np.append(stack, np.eye(NUM_BLOCKS_PER_LAYER)[0])
    else:
        stack[-1, num_blocks_top_layer] = 1

    # also ensure that adding the block to the top won't topple the stack
    return validate_removals(stack, possible_moves)

In [ ]:
# TODO
class Stack:
    def __init__(self, num_players):
        self.players = [Player(self) for _ in num_players]
        self.tower_height = NUM_LAYERS
        self.move_count = 0
        self.boolean_stack = np.ones(self.tower_height, NUM_BLOCKS_PER_LAYER)

    @property
    def toppled(self):
        return False

    @property
    def valid_moves(self):
        return find_valid_moves(self.boolean_stack)

# TODO
class Player:
    def __init__(self, stack):
        self.stack = stack

    def move(self):
        for move in self.stack.valid_moves:
            pass

In [ ]:
stack = Stack()
while not stack.toppled:
    for player in stack.players:
        player.move()
        if stack.toppled:
            break